# Lesson 5: Passive and active monitoring

## Setup

In [ ]:
import pandas as pd

In [ ]:
import whylogs as why

In [ ]:
import helpers

In [ ]:
from langkit import llm_metrics

In [ ]:
llm_metrics.init()

In [ ]:
from whylogs.experimental.core.udf_schema import register_dataset_udf

In [ ]:
from whylogs.experimental.core.udf_schema import udf_schema

In [ ]:
llm_schema = udf_schema()

In [ ]:
llm_logger = why.logger(schema=udf_schema())

In [ ]:
llm_logger = why.logger(
    model = "rolling",
    interval = 1,
    when = "H",
    schema = udf_schema()
)

**Note**: To accessthe WhyLabs platform that was built in the previous lessons: https://hub.whylabsapp.com/resources/model-1/profiles?profile=ref-EBT5yDFL0lyq0r93&sessionToken=session-pPdc5R9m

## Building your own active monitoring guardrails

In [ ]:
import openai

In [ ]:
openai.api_key = helpers.get_openai_key()
openai.base_url = helpers.get_openai_base_url()

In [ ]:
active_llm_logger = why.logger()

In [ ]:
def user_request():
    # Take request
    request = input("\nEnter your desired item to make a recipe" \
                    "(or 'quit'):")
    if request.lower() == "quit":
        raise KeyboardInterrupt()
        
    # Log request
    active_llm_logger.log({"request": request})

    return request

In [ ]:
def prompt_llm(request):
    # Transform prompt
    prompt = f"""Please give me a short recipe for creating"\
    the following item in up to 6 steps. Each step of the recipe "\
    should be summarized in no more than 200 characters."\
    Item: {request}"""

    # Log prompt
    active_llm_logger.log({"prompt": prompt})

    # Collect response from LLM
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [{
            "role": "system",
            "content": prompt
        }]
    )["choices"][0]["message"]["content"]

    # Log response
    active_llm_logger.log({"response": response})

    return response

In [ ]:
def user_reply_success(request,response):
    # Create and print user reply
    reply = f"\nSuccess! Here is the recipe for"\
            f"{request}:\n{response}"
    print(reply)

    #Log reply
    active_llm_logger.log({"reply": reply})

In [ ]:
def user_reply_failure(request = "your request"):
    # Create and print user reply
    reply = ("\nUnfortunately, we are not able to provide a recipe for " \
            f"{request} at this time. Please try Recipe Creator 900 " \
            f"in the future.")
    print(reply)

    #Log reply
    active_llm_logger.log({"reply": reply})

In [ ]:
class LLMApplicationValidationError(ValueError):
    pass

In [ ]:
while True:
    try:
        request = user_request()
        response = prompt_llm(request)
        user_reply_success(request, response)
    except KeyboardInterrupt:
        break
    except LLMApplicationValidationError:
        user_reply_failure(request)
        break

In [ ]:
from whylogs.core.relations import Predicate
from whylogs.core.metrics.condition_count_metric import Condition
from whylogs.core.validators import ConditionValidator

In [ ]:
def raise_error(validator_name, condition_name, value):
    raise LLMApplicationValidationError(
        f"Failed {validator_name} with value {value}."
    )

In [ ]:
low_condition = {"<0.3": Condition(Predicate().less_than(0.3))}

In [ ]:
toxicity_validator = ConditionValidator(
    name = "Toxic",
    conditions = low_condition,
    actions = [raise_error]
)

In [ ]:
refusal_validator = ConditionValidator(
    name = "Refusal",
    conditions = low_condition,
    actions = [raise_error]
)

In [ ]:
llm_validators = {
    "prompt.toxicity": [toxicity_validator],
    "response.refusal_similarity": [refusal_validator]
}

In [ ]:
active_llm_logger = why.logger(
    model = "rolling",
    interval = 5,
    when = "M",
    base_name = "active_llm",
    schema = udf_schema(validators = llm_validators)
)

In [ ]:
active_llm_logger.log(
    {"response":"I'm sorry, but I can't answer that."}
)

 ⚠️ **Disclaimer**: Please be aware that the code may not capture all safety concerns and some undesired responses can still pass through. We encourage you to explore ways in which you can make the monitoring system more robust.

In [ ]:
while True:
    try:
        request = user_request()
        response = prompt_llm(request)
        user_reply_success(request, response)
    except KeyboardInterrupt:
        break
    except LLMApplicationValidationError:
        user_reply_failure(request)
        break